# Testing Agentic Planning with ControlFlow

In [1]:
from typing import List, Literal

import controlflow as cf
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

## Multi-Agent Solution
The flow we're eying atm would be this:
```
A: input P -> B [find Ps without NA children]
B -> C{Any found?}
C -> |yes| D[generate children for these Ps]
D -> E[Classify new children as P or NA]
E -> D
C -> |no| F[Check if each P's direct children achieve its outcome]
F -> |Incomplete Ps| D
F -> |all complete| G[Final Task Tree]
```

But let's build towards this gradually.






### V0.0.3 - add Task classification as Project or Next Action

In [6]:
class TaskNode(BaseModel):
    id: str = Field(description="Unique identifier for the task")
    title: str = Field(description="Title/description of the task")
    type: Literal["project", "next_action"] = Field(
        description="Type of the task, either 'project' or 'next_action'"
    )
    subtasks: List["TaskNode"] = Field(default_factory=list)


TaskNode.model_rebuild()


# Result wrapper for ControlFlow
class TaskResult(BaseModel):
    task: TaskNode

In [8]:
def print_task_nodes(task: TaskNode, indent: int = 0):
    """Print task tree recursively with nice formatting.

    Args:
        task: Task to print
        indent: Current indentation level
    """
    # Print current task with indentation
    classification = "P" if task.type == "project" else "NA"
    print("    " * indent + f"{task.id} - {classification} - {task.title}")

    # Recursively print subtasks
    for subtask in task.subtasks:
        print_task_nodes(subtask, indent + 1)

In [9]:
chunker_instructions = """
You are a Getting Things Done project management expert, specialized into
subdividing big tasks into smaller ones.
"""
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions=chunker_instructions,
)

classifier_instructions = """
You are a Getting Things Done expert at classifying tasks.

A task should be classified as:
- 'project' if it requires multiple next actions to complete
- 'next_action' if it's a single, concrete actionable step that can be done in one go

Example:
- "Get a job" -> project (requires multiple steps)
- "Send email to recruiter" -> next_action (single action)
"""

classifier = cf.Agent(
    name="Task Classifier",
    model="openai/gpt-4o-mini",
    instructions=classifier_instructions,
)


@cf.flow
def chunk_flow_v0_0_3(project: str):
    # Initialize with root project
    root_task_id = "1"
    root_task = TaskNode(id=root_task_id, title=project, type="project")

    with cf.Task(
        "Generate hyrarchical task tree for this project, where each task is a \
            either a project or a next action",
        result_type=TaskResult,
        agents=[chunker, classifier],
    ) as main_task:
        task_tree = cf.run(
            """
            Break given task into subtasks.
            Return a TaskResult containing a TaskNode with:
            - The root task
            - 3-5 subtasks in the subtasks list
            Once you split the task into subtasks, mark this task as complete.
            """,
            context=dict(
                task=root_task,
            ),
            agents=[chunker],
            result_type=TaskResult,
            tools=[main_task.get_success_tool()],
        )

        classified_tree = cf.run(
            """
            Classify each subtask as either 'project' or 'next_action'.
            Return the complete TaskResult with classifications added.
            Once done, use the main task tool to mark the entire process as complete.
            """,
            context=dict(task_tree=task_tree),
            agents=[classifier],
            result_type=TaskResult,
            tools=[main_task.get_success_tool()],
        )

        # tasks = TaskList(tasks=tasks.tasks + new_tasks.tasks)

    return classified_tree.task


tasks = chunk_flow_v0_0_3("Get a job as a genAI engineer")

print(tasks)

Output()

Output()

id='1' title='Get a job as a genAI engineer' type='project' subtasks=[TaskNode(id='1a', title='Update resume with relevant skills and experiences', type='next_action', subtasks=[]), TaskNode(id='1b', title='Research job openings in the genAI field', type='next_action', subtasks=[]), TaskNode(id='1c', title='Prepare for technical interviews by practicing coding questions', type='next_action', subtasks=[]), TaskNode(id='1d', title='Network with industry professionals on LinkedIn', type='next_action', subtasks=[]), TaskNode(id='1e', title='Apply to selected job openings', type='next_action', subtasks=[])]


In [10]:
print_task_nodes(tasks)

1 - P - Get a job as a genAI engineer
    1a - NA - Update resume with relevant skills and experiences
    1b - NA - Research job openings in the genAI field
    1c - NA - Prepare for technical interviews by practicing coding questions
    1d - NA - Network with industry professionals on LinkedIn
    1e - NA - Apply to selected job openings
